# Full Pipeline - Few Images

Assume that we are using the PySpark kernel

## Read files recursively

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

import requests
from PIL import Image
from io import BytesIO

#import matplotlib.pyplot as plt
import numpy as np

In [39]:
# https://sparkbyexamples.com/spark/spark-read-binary-file-into-dataframe/
df = spark.read.format("image").option("recursiveFileLookup", True).load("s3://multimedia-commons/data/images/000/")
#df = spark.read.format("image").option("recursiveFileLookup", True).load("s3://multimedia-commons/data/images/00*")
#df = spark.read.format("image").option("recursiveFileLookup", True).load("s3://multimedia-commons/data/images/{00*,01*}")
#df = spark.read.format("image").option("recursiveFileLookup", True).load("s3://multimedia-commons/data/images/{00*,01*}").persist()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

## Set up ML

Hardcoded values for TensorFlow Hub

In [46]:
model_name = "efficientnetv2-s"
model_handle = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/classification/2'
image_size = 224
dynamic_size = False
labels_file = "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Get classes

In [47]:
#download labels and creates a maps
downloaded_file = tf.keras.utils.get_file("labels.txt", origin=labels_file)

classes = []

with open(downloaded_file) as f:
  labels = f.readlines()
  classes = [l.strip() for l in labels]
print(f'Downloaded {len(classes)} classes')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Downloaded 1001 classes

In [48]:
classifier = hub.load(model_handle)

input_shape = (1, image_size, image_size, 3) #image.shape
warmup_input = tf.random.uniform(input_shape, 0, 1.0)
warmup_logits = classifier(warmup_input).numpy()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Build call to ML within Spark

UDF using PIL - fails

In [81]:
def to_img_np(image):
    image_data = image.data
    h = image.height
    w = image.width
    c = image.nChannels
    img_b = bytes(image_data)
    # https://stackoverflow.com/a/50026948/11262633
    img_pil = Image.frombytes('RGB', (h,w), img_b, 'raw')
    img_np = np.asarray(img_pil)
    return img_pil

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
_udf = F.udf(lambda image: to_img_np(image)) 
df = df.withColumn('img_np',_udf(df_new['image']))
for x in df_show.select('t').collect():
    print(x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o1056.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 55.0 failed 4 times, most recent failure: Lost task 4.3 in stage 55.0 (TID 384) (ip-172-31-18-145.us-east-2.compute.internal executor 1): net.razorvine.pickle.PickleException: failed to __setstate__()
	at net.razorvine.pickle.Unpickler.load_build(Unpickler.java:409)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:234)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:123)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:136)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$6(BatchEvalPythonExec.scala:94)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:490)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	a

### Can call TensorFlow utility functions and return scalar results

In [ ]:
df = spark.read.format("image").option("recursiveFileLookup", True).load("s3://multimedia-commons/data/images/{00*,01*}")

In [55]:
#https://stackoverflow.com/questions/44067861/pyspark-add-a-new-column-with-a-tuple-created-from-columns
from pyspark.sql.functions import struct

def test(data_tuple):
  #classifier = hub.load(model_handle)
  w = data_tuple[0]
  h = data_tuple[1]
  c = data_tuple[2]
  vecs = data_tuple[3]
  # reshape into shape [batch_size, height, width, num_channels]
  img_reshaped = tf.reshape(vecs, [1, h, w, c])
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  image = tf.image.convert_image_dtype(img_reshaped, tf.float32)
  # Run model on image
  #results = classifier(image)
  #probabilities = tf.nn.softmax(results).numpy() 
  #top_5 = tf.argsort(probabilities, axis=-1, direction="DESCENDING")[0][:5].numpy() 
  #return top_5
  return image.shape[0] #probabilities

#_udf = F.udf(lambda image: image.origin)  # Works
#_udf = F.udf(lambda image: to_img_from_spark(image).size) # Fails - returns string ~ '[Ljava.lang.Object;@38ba1a87'
_udf = F.udf(lambda image: test(image)) 
df_new2 = df_new.withColumn('c',struct(df_new.image.width, df_new.image.height, df_new.image.nChannels, df_new.vecs))
df_show = df_new2.withColumn('t',_udf(df_new2['c']))
for x in df_show.select('t').collect():
    print(x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(t='1')
Row(t='1')
Row(t='1')
Row(t='1')
Row(t='1')

### Fails with complex data types

Works if return data as a string

In [83]:
#https://stackoverflow.com/questions/44067861/pyspark-add-a-new-column-with-a-tuple-created-from-columns
from pyspark.sql.functions import struct

def test(data_tuple, return_string=False):
  classifier = hub.load(model_handle)
  w = data_tuple[0]
  h = data_tuple[1]
  c = data_tuple[2]
  vecs = data_tuple[3]
  # reshape into shape [batch_size, height, width, num_channels]
  img_reshaped = tf.reshape(vecs, [1, h, w, c])
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  image = tf.image.convert_image_dtype(img_reshaped, tf.float32)
  # Run model on image
  results = classifier(image)
  probabilities = tf.nn.softmax(results)
  top_5 = tf.argsort(probabilities, axis=-1, direction="DESCENDING")[0][:5].numpy() # Shape (5,)
  #return top_5
  if return_string:
    ret_data = str(top_5)
  else:
    ret_data = top_5
  return ret_data  # str(type(probabilities)) results.numpy().shape[0] 

_udf = F.udf(lambda image: test(image, return_string=True)) 
df_new2 = df_new.withColumn('c',struct(df_new.image.width, df_new.image.height, df_new.image.nChannels, df_new.vecs))
df_show = df_new2.withColumn('t',_udf(df_new2['c']))
for x in df_show.select('t').collect():
    print(x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(t='[478   0   1   2   3]')
Row(t='[427   0   1   2   3]')
Row(t='[693   0   1   2   3]')
Row(t='[237 589 740 585 236]')
Row(t='[478 681 155 469   0]')

Same as above but returning top_5, instead of a string version of it.

In [84]:
_udf = F.udf(lambda image: test(image, return_string=False)) 
df_new2 = df_new.withColumn('c',struct(df_new.image.width, df_new.image.height, df_new.image.nChannels, df_new.vecs))
df_show = df_new2.withColumn('t',_udf(df_new2['c']))
for x in df_show.select('t').collect():
    print(x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o1112.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 57.0 failed 4 times, most recent failure: Lost task 4.3 in stage 57.0 (TID 408) (ip-172-31-18-145.us-east-2.compute.internal executor 1): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.core.multiarray._reconstruct)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:773)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:213)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:123)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:136)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$6(BatchEvalPythonExec.scala:94)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11

In [67]:
df_show.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Pandas >= 0.23.2 must be installed; however, it was not found.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py", line 62, in toPandas
    require_minimum_pandas_version()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/utils.py", line 34, in require_minimum_pandas_version
    "it was not found." % minimum_pandas_version) from raised_error
ImportError: Pandas >= 0.23.2 must be installed; however, it was not found.



In [68]:
df_show.rdd.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 48.0 failed 4 times, most recent failure: Lost task 4.3 in stage 48.0 (TID 308) (ip-172-31-18-145.us-east-2.compute.internal executor 1): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.core.multiarray._reconstruct)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:773)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:213)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:123)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:136)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$6(BatchEvalPythonExec.scala:94)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at 